# Gradient Boosting Classification

In [2]:
from sklearn.datasets import  load_iris

In [3]:
iris = load_iris()
data =  iris.data
target = iris.target

In [4]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3)

In [6]:
from sklearn.ensemble import GradientBoostingClassifier

In [11]:
model = GradientBoostingClassifier()

In [14]:
model.fit(X_train, y_train)

GradientBoostingClassifier()

In [16]:
predictions = model.predict(X_test)

In [18]:
from sklearn.metrics import accuracy_score

In [20]:
accuracy = accuracy_score(y_test, predictions)

In [22]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.9555555555555556


# Hyperparameter Tuning using GridSearchCV

In [25]:
from sklearn.model_selection import GridSearchCV

In [27]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 1.0]
}

In [29]:
grid_search = GridSearchCV(model, param_grid, cv=3)

In [31]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 10], 'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200],
                         'subsample': [0.8, 1.0]})

In [32]:
grid_search.best_params_

{'learning_rate': 0.01,
 'max_depth': 3,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 50,
 'subsample': 1.0}

In [36]:
best_model = grid_search.best_estimator_

In [38]:
best_model.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.01, min_samples_leaf=4,
                           n_estimators=50)

In [40]:
predictions = best_model.predict(X_test)

In [42]:
accuracy = accuracy_score(y_test, predictions)

In [44]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.9333333333333333


# Hyperparameter Tuning using Optuna

In [47]:
import optuna

In [49]:
# Define objective function for Optuna
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300, step=50)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, step=0.05)
    max_depth = trial.suggest_int('max_depth', 3, 15, step=2)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20, step=2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10, step=1)
    subsample = trial.suggest_float('subsample', 0.6, 1.0, step=0.1)

    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        subsample=subsample,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    Accuracy = accuracy_score(y_test, y_pred)
    return Accuracy

In [51]:
# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2025-03-01 14:24:03,624] A new study created in memory with name: no-name-c3da98ab-2ca7-48f5-a4d7-d51650601bf9
C:\Users\suren\anaconda3\Lib\site-packages\optuna\distributions.py:685: UserWarning: The distribution is specified by [0.01, 0.3] and step=0.05, but the range is not divisible by `step`. It will be replaced by [0.01, 0.26].
  warnings.warn(
[I 2025-03-01 14:24:04,051] Trial 0 finished with value: 0.9333333333333333 and parameters: {'n_estimators': 200, 'learning_rate': 0.26, 'max_depth': 11, 'min_samples_split': 14, 'min_samples_leaf': 4, 'subsample': 0.8}. Best is trial 0 with value: 0.9333333333333333.
C:\Users\suren\anaconda3\Lib\site-packages\optuna\distributions.py:685: UserWarning: The distribution is specified by [0.01, 0.3] and step=0.05, but the range is not divisible by `step`. It will be replaced by [0.01, 0.26].
  warnings.warn(
[I 2025-03-01 14:24:04,512] Trial 1 finished with value: 0.9333333333333333 and parameters: {'n_estimators': 300, 'learning_rate': 0.26

In [53]:
study.best_params

{'n_estimators': 150,
 'learning_rate': 0.26,
 'max_depth': 13,
 'min_samples_split': 6,
 'min_samples_leaf': 2,
 'subsample': 1.0}

In [55]:
study.best_value

0.9777777777777777

In [57]:
best_params = study.best_params

In [59]:
best_model1 = GradientBoostingClassifier(**best_params)

In [61]:
best_model1.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.26, max_depth=13, min_samples_leaf=2,
                           min_samples_split=6, n_estimators=150)

In [63]:
predictions = best_model1.predict(X_test)

In [65]:
accuracy = accuracy_score(y_test, predictions)

In [67]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.9777777777777777
